# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning I</font>

## Construindo Um Algoritmo Para Rede Neural Multilayer Perceptron

In [1]:
from IPython.display import Image
Image(url = 'Neuronio01.png')

In [2]:
from IPython.display import Image
Image(url = 'Neuronio02.png')

In [1]:
from IPython.display import Image
Image(url = 'imgs/17-MLP06-01.png')

In [2]:
from IPython.display import Image
Image(url = 'imgs/17-MLP06-02.png')

### Otimização com Gradiente Descendente 

In [6]:
import numpy as np


class Neuronio(object):
    def __init__(self, nodes_entrada = []):
        self.nodes_entrada = nodes_entrada
        self.nodes_saida = []
        self.valor = None
        self.gradientes = {}
        for n in nodes_entrada:
            n.nodes_saida.append(self)

    def forward(self):
        raise NotImplementedError

    def backward(self):
        raise NotImplementedError


class Input(Neuronio):
    def __init__(self):
        Neuronio.__init__(self)

    def forward(self):
        pass

    def backward(self):
        # Um nó de entrada não possui inputs e assim o gradiente (derivada) é zero.
        # A palavra reservada, `self`, é referência a este objeto.
        self.gradientes = {self: 0}
        
        # Os pesos e o bias podem ser entradas, então você precisa somar o gradiente, dos gradientes de saída.
        for n in self.nodes_saida:
            grad_cost = n.gradientes[self]
            self.gradientes[self] += grad_cost 


class Linear(Neuronio):
    def __init__(self, X, W, b):
        Neuronio.__init__(self, [X, W, b])

    def forward(self):
        X = self.nodes_entrada[0].valor
        W = self.nodes_entrada[1].valor
        b = self.nodes_entrada[2].valor
        self.valor = np.dot(X, W) + b

    def backward(self):
        """
        Calcula o gradiente com base nos valores de saída.
        """
        # Inicializa um parcial para cada um dos nodes_entrada.
        self.gradientes = {n: np.zeros_like(n.valor) for n in self.nodes_entrada}
        
        # Percorrendo as saídas. 
        # O gradiente mudará dependendo de cada saída, então os gradientes são somados em todas as saídas.
        for n in self.nodes_saida:
            
            # Obtendo parcial do custo em relação a este nó.
            grad_cost = n.gradientes[self]
            
            # Definindo a parcial da perda em relação às entradas deste nó.
            self.gradientes[self.nodes_entrada[0]] += np.dot(grad_cost, self.nodes_entrada[1].valor.T)
            
            # Definindo a parcial da perda em relação aos pesos deste nó.
            self.gradientes[self.nodes_entrada[1]] += np.dot(self.nodes_entrada[0].valor.T, grad_cost)
            
            # Definindo a parcial da perda em relação ao bias deste nó.
            self.gradientes[self.nodes_entrada[2]] += np.sum(grad_cost, axis = 0, keepdims = False)


class Sigmoid(Neuronio):
    def __init__(self, node):
        Neuronio.__init__(self, [node])

    def _sigmoid(self, x):
        return 1. / (1. + np.exp(-x))

    def forward(self):
        input_value = self.nodes_entrada[0].valor
        self.valor = self._sigmoid(input_value)

    def backward(self):
        """
        Calcula o gradiente usando a derivada da função sigmoide.
        
        O método backward da classe Sigmoid, soma as derivadas (é uma derivada normal quando há apenas uma variável) 
        em relação à única entrada sobre todos os nós de saída.
        """
        # Inicializa os gradientes com 0.
        self.gradientes = {n: np.zeros_like(n.valor) for n in self.nodes_entrada}
        
        # O gradiente mudará dependendo de cada saída e então os gradientes são somados em todas as saídas.
        for n in self.nodes_saida:
            grad_cost = n.gradientes[self]
            sigmoid = self.valor
            self.gradientes[self.nodes_entrada[0]] += sigmoid * (1 - sigmoid) * grad_cost


class MSE(Neuronio):
    def __init__(self, y, a):
        Neuronio.__init__(self, [y, a])

    def forward(self):
        y = self.nodes_entrada[0].valor.reshape(-1, 1)
        a = self.nodes_entrada[1].valor.reshape(-1, 1)

        self.m = self.nodes_entrada[0].valor.shape[0]
        
        # Salva o resultado para o backward.
        self.diff = y - a
        self.value = np.mean(self.diff**2)

    def backward(self):
        """
        Calcula o gradiente do custo.

         Este é o nó final da rede para os nós de saída
        """
        self.gradientes[self.nodes_entrada[0]] = (2 / self.m) * self.diff
        self.gradientes[self.nodes_entrada[1]] = (-2 / self.m) * self.diff


def topological_sort(feed_dict):
    """
    Classifica os nós em ordem topológica usando o Algoritmo de Kahn.

    `Feed_dict`: um dicionário em que a chave é um nó `Input` e o valor é o respectivo feed de valor para esse nó.

    Retorna uma lista de nós ordenados.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.nodes_saida:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.valor = feed_dict[n]

        L.append(n)
        for m in n.nodes_saida:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_and_backward(graph):
    """
   Executa uma passagem para a frente e uma passagem para trás através de uma lista de nós ordenados.

     Argumentos:

         `Graph`: O resultado de `topological_sort`.
    """

    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # O valor negativo no slice permite fazer uma cópia da mesma lista na ordem inversa.
    for n in graph[::-1]:
        n.backward()

### Executando o Grafo

In [7]:
import numpy as np

# Parâmetros de entrada
X, W, b = Input(), Input(), Input()

# Operações
y = Input()
f = Linear(X, W, b)
a = Sigmoid(f)
cost = MSE(y, a)

# Atribuindo valores aos parâmetros
entrada = np.array([[-1., -2.], [-1, -2]])
pesos = np.array([[2.], [3.]])
bias = np.array([-3.])
saida = np.array([1, 2])

# Define o feed_dict
feed_dict = {X: entrada, y: saida, W: pesos, b: bias}

# Ordena as entradas para execução
graph = topological_sort(feed_dict)

# Forward e Backward
forward_and_backward(graph)

# Retorna os gradientes de cada input
gradientes = [t.gradientes[t] for t in [X, y, W, b]]

# print
print(gradientes)

[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]


## Fim